In [ ]:
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import math
import warnings
warnings.filterwarnings('ignore')

In [ ]:
FRAME_SIZE = 2048
HOP_SIZE = 512
SOUNDS_PATH = '../input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files'

In [ ]:
#pr signifies "posterior right"
h_pr_path = SOUNDS_PATH + '/101_1b1_Pr_sc_Meditron.wav' #h means healthy
w_pr_path = SOUNDS_PATH + '/106_2b1_Pr_mc_LittC2SE.wav' #w means wheezes present
c_pr_path = SOUNDS_PATH + '/107_2b4_Pr_mc_AKGC417L.wav' #c means crackles present
w_c_pr_path = SOUNDS_PATH + '/107_2b3_Pr_mc_AKGC417L.wav'#w_c means crackles and wheezes present
easy_ausc_c = '../input/crackles1/crackles.wav' 
easy_ausc_h = '../input/normal1/normal-vesicular.wav'

# Trouble (anomaly wrt BER) is with the following two files
* The two test files below were downloaded from another website for personal experimentation purposes only
* I first downloaded the MP3, then converted to .wav using audacity

Test Files: 
1. easy_ausc_c = '../input/crackles1/crackles.wav' 
2. easy_ausc_h = '../input/normal1/normal-vesicular.wav'

In [ ]:
def plot_spectrogram(Y, sr, hop_length, y_axis="linear"):
    plt.figure(figsize=(25, 10))
    librosa.display.specshow(Y, 
                             sr=sr, 
                             hop_length=hop_length, 
                             x_axis="time", 
                             y_axis=y_axis)
    plt.colorbar(format="%+2.f")

In [ ]:
def calculate_split_frequency_bin(split_frequency, sample_rate, num_frequency_bins):
    """Infer the frequency bin associated to a given split frequency."""
    
    frequency_range = sample_rate / 2
    frequency_delta_per_bin = frequency_range / num_frequency_bins
    split_frequency_bin = math.floor(split_frequency / frequency_delta_per_bin)
    return int(split_frequency_bin)

In [ ]:
def band_energy_ratio(spectrogram, split_frequency, sample_rate):
    """Calculate band energy ratio with a given split frequency."""
    
    split_frequency_bin = calculate_split_frequency_bin(split_frequency, sample_rate, len(spectrogram[0]))
    band_energy_ratio = []
    
    # calculate power spectrogram
    power_spectrogram = np.abs(spectrogram) ** 2
    power_spectrogram = power_spectrogram.T
    
    # calculate BER value for each frame
    for frame in power_spectrogram:
        sum_power_low_frequencies = frame[:split_frequency_bin].sum()
        sum_power_high_frequencies = frame[split_frequency_bin:].sum()
        band_energy_ratio_current_frame = sum_power_low_frequencies / sum_power_high_frequencies
        band_energy_ratio.append(band_energy_ratio_current_frame)
    
    return np.array(band_energy_ratio)

In [ ]:
#takes in a wav file and returns "w", "c", "w and/or c" or "h"
def plot_spec_print_max_ber(wavFilePath):
    s, sr = librosa.load(wavFilePath)
    s_spec = librosa.stft(s, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)
    ber_s = band_energy_ratio(s_spec, 64, sr)
    ber_s_no_nan = ber_s[~np.isnan(ber_s)] #because some values turn into 'nan' (div by 0)
    max_ber = max(ber_s_no_nan)
    Y_s = librosa.power_to_db(np.abs(s_spec) ** 2)
    plot_spectrogram(Y_s, sr, HOP_SIZE, y_axis = "log")    
    print('max_ber: ',max_ber)


In [ ]:
soundPaths = [h_pr_path, w_pr_path, c_pr_path, w_c_pr_path, easy_ausc_c, easy_ausc_h]
for soundFile in soundPaths: 
    plot_spec_print_max_ber(soundFile)